In [35]:
import pandas as pd
import re
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.neural_network import MLPClassifier

In [36]:
# Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [37]:
# Improved Text Preprocessing Function
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters
    words = text.split()
    words = [lemmatizer.lemmatize(stemmer.stem(word)) for word in words if word not in stop_words]
    return ' '.join(words).strip()

In [38]:
# Load dataset
df = pd.read_csv("sentimentdataset.csv")
df = df[["Text", "Sentiment"]]
df["Sentiment"] = df["Sentiment"].str.strip()
df["ProcessedText"] = df["Text"].apply(preprocess_text)

In [39]:
# Encode labels
label_encoder = LabelEncoder()
df["SentimentEncoded"] = label_encoder.fit_transform(df["Sentiment"])

# Remove rare classes
class_counts = df["SentimentEncoded"].value_counts()
rare_classes = class_counts[class_counts < 2].index
df = df[~df["SentimentEncoded"].isin(rare_classes)]

In [40]:
# Split dataset
X = df["ProcessedText"]
y = df["SentimentEncoded"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [41]:
# Convert text to numerical features using TF-IDF
vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=30000, min_df=3, stop_words="english", sublinear_tf=True)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [42]:
# Check class distribution before SMOTE
print("Class distribution before SMOTE:\n", y_train.value_counts())

# Ensure SMOTE doesn't fail due to small class sizes
min_class_size = y_train.value_counts().min()
k_neighbors = min(5, min_class_size - 1)  # Ensures k < class size

Class distribution before SMOTE:
 SentimentEncoded
146    36
116    35
76     29
40     15
135    14
       ..
70      2
53      2
157     2
20      2
43      2
Name: count, Length: 112, dtype: int64


In [43]:

# Apply SMOTE with reduced k_neighbors
smote = SMOTE(random_state=42, k_neighbors=k_neighbors)
X_train_tfidf, y_train = smote.fit_resample(X_train_tfidf, y_train)

# Check class distribution after SMOTE
print("Class distribution after SMOTE:\n", pd.Series(y_train).value_counts())

Class distribution after SMOTE:
 SentimentEncoded
136    36
41     36
93     36
184    36
24     36
       ..
70     36
53     36
157    36
20     36
43     36
Name: count, Length: 112, dtype: int64


In [44]:
# # Define parameter grid for MLP (Added more layers & optimizers)
# param_grid = {
#     'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 100)],  # More architectures
#     'activation': ['relu', 'tanh'],  # Activation functions
#     'solver': ['adam', 'sgd'],  # Added SGD optimizer
#     'alpha': [0.0001, 0.001],  # L2 regularization
#     'learning_rate': ['constant', 'adaptive'],  # Learning rate strategy
#     'batch_size': [32],  # Mini-batch training
#     'max_iter': [500]  # Increased training iterations
# }

# # Perform Grid Search with Cross Validation
# mlp = MLPClassifier(random_state=42)
# grid_search = GridSearchCV(mlp, param_grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=2)
# grid_search.fit(X_train_tfidf, y_train)



In [45]:
# Best model

best_mlp = MLPClassifier(
    activation='relu',
    alpha=0.001,
    batch_size=32,
    hidden_layer_sizes=(100, 100),
    learning_rate='constant',
    max_iter=500,
    solver='adam',
    random_state=42
)
best_mlp.fit(X_train_tfidf, y_train)

MLPClassifier(alpha=0.001, batch_size=32, hidden_layer_sizes=(100, 100),
              max_iter=500, random_state=42)

In [46]:
# Predict with the best model
y_pred = best_mlp.predict(X_test_tfidf)

# Evaluate performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Best Accuracy: {accuracy:.4f}")
print("Classification Report:\n", classification_report(y_test, y_pred))

Best Accuracy: 0.6107
Classification Report:
               precision    recall  f1-score   support

           0       0.67      1.00      0.80         2
           1       0.00      0.00      0.00         1
           2       1.00      1.00      1.00         1
           3       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         1
           8       1.00      1.00      1.00         1
           9       0.00      0.00      0.00         1
          11       0.00      0.00      0.00         1
          15       1.00      1.00      1.00         1
          17       1.00      0.50      0.67         2
          18       1.00      1.00      1.00         1
          19       0.50      1.00      0.67         1
          20       0.00      0.00      0.00         1
          21       0.50      1.00      0.67         1
          24       0.00      0.00      0.00         1
          26       1.00      1.00      1.00         1
          27       0.00      0.00  

C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [47]:
# Define DNN model
def build_dnn_model(input_dim, num_classes):
    model = Sequential([
        Dense(512, activation='relu', input_shape=(input_dim,)),
        Dropout(0.3),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [48]:
# Prepare DNN input data
num_classes = len(label_encoder.classes_)
y_train_dnn = to_categorical(y_train, num_classes=num_classes)
y_test_dnn = to_categorical(y_test, num_classes=num_classes)


In [49]:
# Build and train the DNN
input_dim = X_train_tfidf.shape[1]
dnn_model = build_dnn_model(input_dim, num_classes)
dnn_model.fit(X_train_tfidf.toarray(), y_train_dnn, epochs=30, batch_size=32, validation_split=0.1)

# Predictions
y_pred_dnn = np.argmax(dnn_model.predict(X_test_tfidf.toarray()), axis=1)
print("DNN Accuracy:", accuracy_score(y_test, y_pred_dnn))
print("DNN Classification Report:\n", classification_report(y_test, y_pred_dnn))

C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
114/114 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step - accuracy: 0.0409 - loss: 4.9031 - val_accuracy: 0.0000e+00 - val_loss: 9.9752
Epoch 2/30
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.6482 - loss: 1.6717 - val_accuracy: 0.0272 - val_loss: 9.2464
Epoch 3/30
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9002 - loss: 0.4630 - val_accuracy: 0.1163 - val_loss: 5.8254
Epoch 4/30
114/114 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.9425 - loss: 0.2578 - val_accuracy: 0.1906 - val_loss: 3.2594
Epoch 5/30
114/114 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.9614 - loss: 0.1582 - val_accuracy: 0.4876 - val_loss: 1.6697
Epoch 6/30
114/114 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.9704 - loss: 0.1132 - val_accuracy: 0.7673 - val_loss: 1.0252
Epoch 7/30
114/114 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.9750 - loss: 0.0910 - val_accuracy: 0.8713 - val_loss: 0.7499
Epoch 8/30
114/114 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.9814 - loss: 0.0737 - val_

C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
